In [42]:
import madmom
import pickle
import numpy as np
import tensorflow as tf

In [32]:
# Workaround for opening Python2 pickle object
with open('datasets/madmom_models-master/onsets/2013/onsets_cnn.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    p = u.load()



In [41]:
# Batch Normalization layer
lay = p.layers[0]
print(dir(lay))
print("Activation_fn: ", lay.activation_fn)
print("Beta: ", lay.beta)
print("Gamma: ", lay.gamma)
print("Inv std: ", lay.inv_std.shape)
print("Mean: ", lay.mean.shape)

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'activate', 'activation_fn', 'beta', 'gamma', 'inv_std', 'mean', 'reset']
Activation_fn:  <function linear at 0x11cee9a60>
Beta:  0
Gamma:  1
Inv std:  (80, 3)
Mean:  (80, 3)


In [65]:
# Convolutional layer
lay = p.layers[1]
print("Act fn: ", lay.activation_fn)
print("Weights: ", lay.weights.shape)
print("Bias: ", lay.bias.shape)
print("Pad: ", lay.pad)
print("stride: ",lay.stride)

print(np.transpose(lay.weights, axes=[2,3,0,1]).shape)

Act fn:  <function tanh at 0x11cee9b80>
Weights:  (3, 10, 7, 3)
Bias:  (10,)
Pad:  valid
stride:  1
(7, 3, 3, 10)


In [75]:
lay = p.layers[3]
print("Act fn: ", lay.activation_fn)
print("Weights: ", lay.weights.shape)
print("Bias: ", lay.bias.shape)
print("Pad: ", lay.pad)
print("stride: ",lay.stride)

print(np.transpose(lay.weights, axes=[2,3,0,1]).shape)


Act fn:  <function tanh at 0x11cee9b80>
Weights:  (10, 20, 3, 3)
Bias:  (20,)
Pad:  valid
stride:  1
(3, 3, 10, 20)


In [84]:
#Pooling layers
p.layers[2].stride, p.layers[2].size, p.layers[4].stride, p.layers[4].size

(array([1, 3]), array([1, 3]), array([1, 3]), array([1, 3]))

In [104]:
lay = p.layers[6]
print("Act fn: ", lay.activation_fn)
print("Weights: ", lay.weights.shape)
print("Bias: ", lay.bias.shape)


Act fn:  <function sigmoid at 0x11cee9c10>
Weights:  (1120, 256)
Bias:  (256,)


1120

In [108]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(80, 15, 3)),
    tf.keras.layers.Permute((2,1,3)),
    tf.keras.layers.Conv2D(
        activation = 'tanh',
        filters = 10,
        kernel_size = (7,3),
        strides = 1
    ),
    tf.keras.layers.MaxPooling2D(
         pool_size=(1, 3), 
            strides=(1,3)
    ),
    tf.keras.layers.Conv2D(
        activation = 'tanh',
        filters = 20,
        kernel_size = (3,3),
        strides = 1
    ),  
    tf.keras.layers.MaxPooling2D(
         pool_size=(1, 3), 
            strides=(1,3)
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'sigmoid'),
    tf.keras.layers.Dense(1, activation = 'sigmoid', 
     )
])

In [110]:
model.layers[1].set_weights([
    np.transpose(p.layers[1].weights, [2,3,0,1]), 
    p.layers[1].bias
])

model.layers[3].set_weights([
    np.transpose(p.layers[3].weights, [2,3,0,1]), 
    p.layers[3].bias
])

model.layers[6].set_weights([
    p.layers[6].weights, 
    p.layers[6].bias
])

model.layers[7].set_weights([
    p.layers[7].weights, 
    p.layers[7].bias
])

[<tf.Variable 'dense_10/kernel:0' shape=(1120, 64) dtype=float32, numpy=
 array([[ 0.01096088, -0.06231456,  0.05448451, ...,  0.05413527,
          0.04888607, -0.03124151],
        [ 0.06031195,  0.04046613, -0.02071594, ...,  0.02188477,
          0.05278087,  0.03738573],
        [-0.00490151,  0.02751981, -0.05178542, ..., -0.05709495,
         -0.05234237, -0.05252508],
        ...,
        [ 0.04643755, -0.03966928,  0.05653187, ..., -0.00513978,
         -0.06503255, -0.06670948],
        [ 0.06526931,  0.05491904,  0.0332076 , ..., -0.04825261,
          0.0553588 , -0.03803964],
        [-0.02238995,  0.00387419, -0.0350247 , ..., -0.02247724,
         -0.04711654,  0.06887134]], dtype=float32)>,
 <tf.Variable 'dense_10/bias:0' shape=(64,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.